# YOLO를 해봅시다

In [1]:
import cv2
import argparse
import numpy as np
import os.path
from matplotlib import pyplot as plt
%matplotlib inline

##  기본 cv 함수셋팅

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

from bokeh.plotting import figure #pip install bokeh
from bokeh.io import output_notebook, show, push_notebook


output_notebook()


def imshow(tit, image) :
    plt.title(tit)    
    if len(image.shape) == 3 :
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else :
        plt.imshow(image, cmap="gray")
    plt.show()
    
    
def create_win(frames, scale=1.0) :    
    global myImage
    
    all = []
    for f in frames :
        if len(f.shape ) !=  3 : f = cv2.cvtColor(f, cv2.COLOR_GRAY2BGR)
        all.append(f)
    frame = np.vstack(all)
    
    fr=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
    fr=cv2.flip(fr, -1) # because Bokeh flips vertically
    width=fr.shape[1]
    height=fr.shape[0]    

    p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=int(width*scale), height=int(height*scale))    
    myImage = p.image_rgba(image=[fr], x=0, y=0, dw=width, dh=height)
    show(p, notebook_handle=True)   
    
    
def update_win(frames) :
    
    all = []
    for f in frames :
        #print(len(f.shape))
        if len(f.shape ) !=  3 : f = cv2.cvtColor(f, cv2.COLOR_GRAY2BGR)
        all.append(f)
    frame = np.vstack(all)
    
    fr=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    fr=cv2.flip(fr, 0)
    myImage.data_source.data['image']=[fr]
    push_notebook()

Loading BokehJS ...

##  YOLO 셋팅

In [3]:
# Initialize the parameters
confThreshold = 0.3  #Confidence threshold, 오브젝트일 확률
nmsThreshold = 0.4   #Non-maximum suppression threshold , 거리가 가까운애들은 하나의 그룹으로 보겟다, 주변보다 상대적으로 값이 높거나 낮거나 하는 애를 뽑겠다.

#YOLO v3는 영상을 무조건 416 사이즈로 변경
inpWidth = 416       #Width of network's input image
inpHeight = 416      #Height of network's input image

# Load names of classes
classesFile = "cfg/one.names" 
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
print(classes)

# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "cfg/one.cfg" #네트워크 구조체(CNN 필터 값들)가 포함되어있다, #컨피그 상에서 실제로 Yolo 관련 컨피그가 3개가 있다.
modelWeights = "cfg/one_class_v1_1400.weights"

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

['fainting_people']


In [13]:
# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames() #네트웍의 모든 이름을 가져오는 함수 ,총 갯수는 254개, 실제 레이어는 100 몇개,
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()] #실제 output 위치는 getUnconnectedOutLayers에서 1뺌

# Draw the predicted bounding box
#사각형으로 바운딩한 곳을 표시
def drawPred(frame, classId, conf, left, top, right, bottom): #클래스ID, 클래스에 대한 일치 확률,사각형 정보
    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    
    label = '%.2f' % conf
        
    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    #Display the label at the top of the bounding box
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv2.FILLED)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)

# Remove the bounding boxes with low confidence using non-maxima suppression
#
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:] #확률 80개의 값을 가져옴
            classId = np.argmax(scores) #확률에서 높은거 가져옴
            confidence = scores[classId] #확률 값 가져옴
            if confidence > confThreshold: # confThreshold보다 더 높을 경우, 박스를 만듬
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
        
    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    
#     indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
#     #같은 포인트에서 다수의 바운딩 박스가 생성되어 있는걸 좀더 정확하게 판별해서 최소화 해줌
    
#     for i in indices:
#         i = i[0] #2차원 행렬이므로 
#         box = boxes[i]
#         left = box[0]
#         top = box[1]
#         width = box[2]
#         height = box[3]
#         drawPred(frame, classIds[i], confidences[i], left, top, left + width, top + height)

## 특정 디렉토리에 이미지 불러오기

In [ ]:
# #시스템적으로 맞게 변경한 코드
# from imutils import paths

# imagePaths = list(paths.list_images("pic")) # 이미지파일의 목록만 알려주는 함수,known 폴더 안(하위폴더 포함)에 
 
# knownEncodings = []
# knownNames = []

# for (i, imagePath) in enumerate(imagePaths):    
#     #imagePath.split(os.path.sep) #['pic', 'oh', 'oh.jpg'] , 이미지의 full path가 리스트 형식으로 나온다.
#     name = imagePath.split(os.path.sep)[-2] 
#     print(f"{name}   -   {imagePath}") 
#     image = cv2.imread(imagePath)
#     boxes = face_recognition.face_locations(image)
#     encodings = face_recognition.face_encodings(image, boxes) 

#     for encoding in encodings:        
#         knownEncodings.append(encoding)
#         knownNames.append(name)
        
# import pickle
# data = {"encodings": knownEncodings, "names": knownNames}
# f = open("pic.bin", "wb") #open 함수를 통해 파일오픈, 쓰기(w)와 바이너리(b) 모드
# f.write(pickle.dumps(data)) # 딕셔너리를 known.bin 이라는 파일에 저장
# f.close()        

## YOLO video

In [14]:
import time
print("yolo VIDEO process")
font = cv2.FONT_HERSHEY_PLAIN
cap = cv2.VideoCapture("video/517-3_cam03_swoon03_place04_night_spring.mp4")


frame_id = 0

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
print('width :%d, height : %d' % (frame_width, frame_height))
fourcc = cv2.VideoWriter_fourcc(*'XVID') #운영체제별로 지원하는게 다름 확인필요
out = cv2.VideoWriter('video_result/517-3_cam03_swoon03_place04_night_spring_1800.avi', fourcc, 29.0, (frame_width, frame_height)) 

i = 0
prevTime = 0
while(True):
    hasFrame, frame = cap.read()    # Read 결과와 frame
    curTime= time.time()
    
    if not hasFrame:
        break
    #print(hasFrame)
    #print(frame)
    print(i)
    if(hasFrame) :
        if i > 100:
            print(i, ': caputer ..')
            blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)
            net.setInput(blob)
            outs = net.forward(getOutputsNames(net))
            postprocess(frame, outs)
            #cv2.imshow('frame_color', frame)    # 컬러 화면 출력
            #cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)
            #out.write(frame)
        if i > 300:
            break
    sec = curTime - prevTime
    prevTime = curTime
    fps = 1/sec
    print("Time: ",sec)
    print("FPS: ",fps)
    i += 1

cap.release()
cv2.destroyAllWindows()

yolo VIDEO process
width :3840, height : 2160
0
Time:  1583907853.689797
FPS:  6.313498589393614e-10
1
Time:  0.018413543701171875
FPS:  54.30785166770251
2
Time:  0.0183718204498291
FPS:  54.43118730290564
3
Time:  0.018551349639892578
FPS:  53.90443387739365
4
Time:  0.018327951431274414
FPS:  54.56147151795819
5
Time:  0.01845836639404297
FPS:  54.175975200206665
6
Time:  0.018483638763427734
FPS:  54.10190129762918
7
Time:  0.01836371421813965
FPS:  54.4552146761357
8
Time:  0.018316268920898438
FPS:  54.596271998333854
9
Time:  0.01831364631652832
FPS:  54.60409045343887
10
Time:  0.01855754852294922
FPS:  53.88642787399147
11
Time:  0.018275976181030273
FPS:  54.71663948861784
12
Time:  0.018308162689208984
FPS:  54.62044537049095
13
Time:  0.01828742027282715
FPS:  54.68239834165548
14
Time:  0.01840686798095703
FPS:  54.32754779545101
15
Time:  0.018545150756835938
FPS:  53.92245191813226
16
Time:  0.01855015754699707
FPS:  53.90789795000321
17
Time:  0.0183107852935791
FPS:  5

Time:  0.30980730056762695
FPS:  3.227812895847859
139
139 : caputer ..
Time:  0.31026506423950195
FPS:  3.2230505953067055
140
140 : caputer ..
Time:  0.30137085914611816
FPS:  3.3181708504708314
141
141 : caputer ..
Time:  0.30442118644714355
FPS:  3.2849224841111027
142
142 : caputer ..
Time:  0.3002510070800781
FPS:  3.3305466973281326
143
143 : caputer ..
Time:  0.3150944709777832
FPS:  3.173651371592961
144
144 : caputer ..
Time:  0.29517674446105957
FPS:  3.387800762644167
145
145 : caputer ..
Time:  0.3006017208099365
FPS:  3.3266609296367826
146
146 : caputer ..
Time:  0.2995913028717041
FPS:  3.3378806073961247
147
147 : caputer ..
Time:  0.30115294456481934
FPS:  3.3205718823206216
148
148 : caputer ..
Time:  0.30399584770202637
FPS:  3.289518615333818
149
149 : caputer ..
Time:  0.3049933910369873
FPS:  3.278759571149945
150
150 : caputer ..
Time:  0.29949951171875
FPS:  3.3389036070919094
151
151 : caputer ..
Time:  0.3049495220184326
FPS:  3.279231242538413
152
152 : capu

Time:  0.3004753589630127
FPS:  3.328059922953935
254
254 : caputer ..
Time:  0.298983097076416
FPS:  3.3446706846588508
255
255 : caputer ..
Time:  0.29094648361206055
FPS:  3.4370582094175455
256
256 : caputer ..
Time:  0.30182862281799316
FPS:  3.3131383984182765
257
257 : caputer ..
Time:  0.3015782833099365
FPS:  3.3158886277374457
258
258 : caputer ..
Time:  0.30640435218811035
FPS:  3.2636612138788146
259
259 : caputer ..
Time:  0.3013739585876465
FPS:  3.3181367251715512
260
260 : caputer ..
Time:  0.3041207790374756
FPS:  3.288167297101307
261
261 : caputer ..
Time:  0.29537487030029297
FPS:  3.3855283592112952
262
262 : caputer ..
Time:  0.29761290550231934
FPS:  3.3600693434720923
263
263 : caputer ..
Time:  0.2953951358795166
FPS:  3.3852960950848967
264
264 : caputer ..
Time:  0.30457043647766113
FPS:  3.283312758667388
265
265 : caputer ..
Time:  0.3007075786590576
FPS:  3.325489847842513
266
266 : caputer ..
Time:  0.29976725578308105
FPS:  3.3359213880372063
267
267 : c

## YOLO 실시하는 함수

In [12]:
def runyolo(name,imagePath):
    print("YOLO GPU process")
    cap = cv2.VideoCapture(imagePath)

    hasFrame, frame = cap.read()
    blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)
    net.setInput(blob)
    outs = net.forward(getOutputsNames(net))
    postprocess(frame, outs)

    #imshow("", frame)
    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv2.getTickFrequency())
    print(f"{name}: {label}")
    cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
    cv2.imwrite("result/"+name+".jpg", frame) 

## 실행

In [13]:
#시스템적으로 맞게 변경한 코드
from imutils import paths

imagePaths = list(paths.list_images("pic")) # 이미지파일의 목록만 알려주는 함수,known 폴더 안(하위폴더 포함)에 
 
knownEncodings = []
knownNames = []

for (i, imagePath) in enumerate(imagePaths):    
    #imagePath.split(os.path.sep) #['pic', 'oh', 'oh.jpg'] , 이미지의 full path가 리스트 형식으로 나온다.
    name = imagePath.split(os.path.sep)[-2] 
    print(f"{name}   -   {imagePath}")
    knownNames.append(name)
    knownEncodings.append(imagePath)
    runyolo(name+str(i), imagePath)
    
import pickle
data = {"encodings": knownEncodings, "names": knownNames}
f = open("pic.bin", "wb") #open 함수를 통해 파일오픈, 쓰기(w)와 바이너리(b) 모드
f.write(pickle.dumps(data)) # 딕셔너리를 known.bin 이라는 파일에 저장
f.close()        #시스템적으로 맞게 변경한 코드

fainting_people   -   pic/fainting_people/116-1_cam02_swoon01_place03_day_spring.mp4_snapshot_04.18_[2020.03.08_00.29.45].jpg
YOLO GPU process
fainting_people0: Inference time: 142.31 ms
fainting_people   -   pic/fainting_people/105-2_cam03_swoon01_place05_night_winter.mp4_snapshot_03.56_[2020.03.08_00.23.24].jpg
YOLO GPU process
fainting_people1: Inference time: 135.80 ms
fainting_people   -   pic/fainting_people/105-4_cam03_swoon01_place05_night_winter.mp4_snapshot_04.01_[2020.03.08_00.25.27].jpg
YOLO GPU process
fainting_people2: Inference time: 140.41 ms
fainting_people   -   pic/fainting_people/240-3_cam01_swoon01_place02_night_winter.mp4_snapshot_04.59_[2020.03.08_00.37.22].jpg
YOLO GPU process
fainting_people3: Inference time: 138.39 ms
fainting_people   -   pic/fainting_people/240-2_cam03_swoon01_place02_night_summer.mp4_snapshot_02.52_[2020.03.08_00.36.27].jpg
YOLO GPU process
fainting_people4: Inference time: 392.38 ms
fainting_people   -   pic/fainting_people/240-1_cam01_swo